# Bifurcation Analysis

In [3]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [4]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')

In [5]:
DSamp = data['DSamp']

triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0] 
fsOld = DSamp[0][0][3][0][0] 
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

In [6]:
import PyDSTool as dst
from PyDSTool import args

# Set up the parameters
par_args = {'Cm': 1.0 * 10**-6, 'g_Na': 120.0 * 10**-3, 'g_K': 36.0 * 10**-3, 
            'g_L': 0.3 * 10**-3, 'E_Na': 50.0, 'E_K': -77.0, 'E_L': -54.387,
            'I': 10 * 10**-6}

# Set up the initial conditions
ic_args = {'V': -65, 'm': 0.05, 'h': 0.6, 'n': 0.32}

# Set up the differential equations
varspecs = {"V": "(I - g_Na*m**3*h*(V - E_Na) - g_K*n**4*(V - E_K) - g_L*(V - E_L)) / Cm",
            "m": "alpha_m*(1.0-m) - beta_m*m",
            "h": "alpha_h*(1.0-h) - beta_h*h",
            "n": "alpha_n*(1.0-n) - beta_n*n"}

# Set up the auxiliary functions
fnspecs = {"alpha_m": (['V'], "0.1*(V+40.0)/(1.0 - exp(-(V+40.0) / 10.0))"),
            "beta_m": (['V'], "4.0*exp(-(V+65.0) / 18.0)"),
            "alpha_h": (['V'], "0.07*exp(-(V+65.0) / 20.0)"),
            "beta_h": (['V'], "1.0/(1.0 + exp(-(V+35.0) / 10.0))"),
            "alpha_n": (['V'], "0.01*(V+55.0)/(1.0 - exp(-(V+55.0) / 10.0))"),
            "beta_n": (['V'], "0.125*exp(-(V+65) / 80.0)")}

# Set up the system
HH = dst.Vode_ODEsystem(dst.args(name='HH', pars=par_args, varspecs=varspecs, 
                                 ics=ic_args, fnspecs=fnspecs))

# Set up the bifurcation analysis
PC = dst.ContClass(HH)

# Define the bifurcation parameter and range
PCargs = dst.args(name='PCargs', type='EP-C', independentvarname='I',
                  MaxNumPoints=1000, MaxStepSize=0.1, MinStepSize=1e-5, 
                  StepSize=2e-2, LocBifPoints='all', SaveEigen=True)
PCargs.freepars = ['I']
PCargs.MaxNumPoints = 500

# Perform the bifurcation analysis
PC.newCurve(PCargs)
PC.curves['PCargs'].forward()

# Plot the bifurcation diagram
PC.display(('I', 'V'), stability=True, figure=1)

TypeError: can't multiply sequence by non-int of type 'method'

In [ ]:
import PyDSTool as dst
import numpy as np
import matplotlib.pyplot as plt

# Define the EEG model
def eeg_model(x, t, p):
    # Define the equations that govern the dynamics of the EEG model
    # Modify these equations to fit your specific model
    dx = x[0] * (p['a'] - x[0]) * (x[0] - 1) - x[1]
    dy = p['c'] * (x[0] - p['b'] * x[1])
    return np.array([dx, dy])

# Set the parameter ranges and values for bifurcation analysis
# Modify these parameter ranges and values to fit your specific model
pars = {
    'a': 0,  # Value for parameter a
    'b': 0,  # Value for parameter b
    'c': 0   # Value for parameter c
}

# Define the PyDSTool model
DSargs = dst.args()
DSargs.name = 'EEG'
DSargs.varspecs = {
    'x': 'x * (a - x) * (x - 1) - y',
    'y': 'c * (x - b * y)',
}
DSargs.pars = pars
DSargs.tdata = [0, time[-1]]  # Time domain for simulation
DSargs.algparams = {'max_pts': 10000, 'maxevtpts': 1000}

# Create a PyDSTool Generator for bifurcation analysis
generator = dst.Generator.Vode_ODEsystem(DSargs)

# Create a PyDSTool Continuation class for bifurcation analysis
PC = dst.ContClass(generator)

# Set up the bifurcation diagram specifications
PCargs = dst.args(name='EQ1', type='EP-C', freepars=['a'], MaxNumPoints=200, MaxStepSize=0.1)
PC.newCurve(PCargs)

# Generate the bifurcation diagram
PC['EQ1'].forward()

# Plot the bifurcation diagram
plt.figure(figsize=(8, 6))
PC.display(('a', 'x'))
plt.xlabel('Parameter a')
plt.ylabel('EEG State Variable x')
plt.title('Bifurcation Diagram for EEG Model')
plt.grid(True)
plt.show()

In [ ]:
# Define the differential equations
def vanderpol_rhs(x, y, mu):
    dxdt = y
    dydt = mu * (1 - x**2) * y - x
    return (dxdt, dydt)

# Parameters for the Van der Pol oscillator
mu = 2.0

In [ ]:
# Define the PyDSTool model
DSargs = dst.args(name='vanderpol')
DSargs.pars = {'mu': mu}
DSargs.varspecs = {'x': 'y', 'y': 'mu * (1 - x**2) * y - x'}
DSargs.ics = {'x': 1, 'y': 1}
DSargs.tdomain = [0, 100]

ode = dst.Generator.Vode_ODEsystem(DSargs)

In [ ]:
# Bifurcation analysis
PC = dst.ContClass(ode)
name1 = 'EQ1'
PCargs = dst.args(name=name1)
PCargs.type = 'EP-C'
PCargs.freepars = ['mu']
PCargs.StepSize = 1e-3
PCargs.MaxNumPoints = 800
PCargs.MaxStepSize = 1e-2
PCargs.MinStepSize = 1e-6
PCargs.MaxTestIters = 40
PCargs.LocBifPoints = 'all'
PCargs.SaveEigen = True
PCargs.StopAtPoints = ['B']

PC.newCurve(PCargs)
PC.curves[name1].backward()
PC.curves[name1].forward()

PC.display(['mu'], stability=True, linewidth=2)
plt.xlabel('mu')
plt.ylabel('x')
plt.title('Bifurcation Diagram')
plt.grid()
plt.show()